In [5]:
# loading GWSNR class from the gwsnr package
from gwsnr import GWSNR
import numpy as np

# initializing the GWSNR class with default parameters
gwsnr = GWSNR(snr_type='interpolation_aligned_spins')

# signal-to-noise ratio for a binary black hole merger with masses 10, 10 Msun at a luminosity distance of 100 Mpc
snrs = gwsnr.snr(mass_1=np.array([30.0]), mass_2=np.array([30.0]), luminosity_distance=np.array([1000.0]), a_1=np.array([0.0]), a_2=np.array([0.0]))

# print the signal-to-noise ratio
print('Computed SNRs:\n', snrs)


Initializing GWSNR class...

psds not given. Choosing bilby's default psds
Interpolator will be loaded for L1 detector from ./interpolator_pickle/L1/partialSNR_dict_1.pickle
Interpolator will be loaded for H1 detector from ./interpolator_pickle/H1/partialSNR_dict_1.pickle
Interpolator will be loaded for V1 detector from ./interpolator_pickle/V1/partialSNR_dict_1.pickle

Chosen GWSNR initialization parameters:

npool:  4
snr type:  interpolation_aligned_spins
waveform approximant:  IMRPhenomD
sampling frequency:  2048.0
minimum frequency (fmin):  20.0
mtot=mass1+mass2
min(mtot):  2.0
max(mtot) (with the given fmin=20.0): 184.98599853446768
detectors:  ['L1', 'H1', 'V1']
psds:  [PowerSpectralDensity(psd_file='None', asd_file='/Users/phurailatpamhemantakumar/anaconda3/envs/ler/lib/python3.10/site-packages/bilby/gw/detector/noise_curves/aLIGO_O4_high_asd.txt'), PowerSpectralDensity(psd_file='None', asd_file='/Users/phurailatpamhemantakumar/anaconda3/envs/ler/lib/python3.10/site-packages/b

In [6]:
# gerneral case, random parameters
# chirp mass can go upto only 95 if f_min=20. to get non zero SNR
nsamples = 1000
chirp_mass = np.linspace(5,60,nsamples)
mass_ratio = np.random.uniform(0.2,1,size=nsamples)
param_dict = dict(
    mass_1 = (chirp_mass*(1+mass_ratio)**(1/5))/mass_ratio**(3/5),
    mass_2 = chirp_mass*mass_ratio**(2/5)*(1+mass_ratio)**(1/5),
    # Fix luminosity distance
    luminosity_distance = 500*np.ones(nsamples),
    # Randomly sample everything else:
    theta_jn = np.random.uniform(0,2*np.pi, size=nsamples),
    ra = np.random.uniform(0,2*np.pi, size=nsamples), 
    dec = np.random.uniform(0,np.pi, size=nsamples), 
    psi = np.random.uniform(0,2*np.pi, size=nsamples),
    phase = np.random.uniform(0,2*np.pi, size=nsamples),
    geocent_time = 1246527224.169434*np.ones(nsamples),
    a_1 = np.random.uniform(-0.8,0.8, size=nsamples),
    a_2 = np.random.uniform(-0.8,0.8, size=nsamples),
)

In [3]:
## aligned spins
# numba njit: time=9.55 s; jax jit: time= 85.1 ms
## no spins
# numba njit: time=547 ms; jax jit: time= 8.11 ms
%timeit snrs = gwsnr.snr_with_interpolation(**param_dict)

8.11 ms ± 278 μs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
snrs = gwsnr.snr_with_interpolation(**param_dict)